Комп'ютерний практикум 1
0. Уважно прочитати методичні вказівки до виконання комп’ютерного практикуму.
1. Написати програми для підрахунку частот букв і частот біграм в тексті, а також підрахунку H1 та H2 за безпосереднім означенням. Підрахувати частоти букв та біграм, а також значення H1 та H2 на довільно обраному тексті російською мовою достатньої довжини (щонайменше 1Мб), де імовірності замінити відповідними частотами. Також одержати значення H1 та H2 на тому ж тексті, в якому вилучено всі пробіли.
2. За допомогою програми CoolPinkProgram оцінити значення (10) H , (20) H , (30) H .
3. Використовуючи отримані значення ентропії, оцінити надлишковість російської мови в різних моделях джерела.

In [37]:
import re # regex
import math
from collections import Counter

In [38]:
def filter_text(text: str,
                keep_space: bool = True) -> str:
    text = text.lower()

    if keep_space:
        filtered = re.sub(r'[^а-яё\s]', ' ', text)
    else:
        filtered = re.sub(r'[^а-яё]', '', text)

    if keep_space:
        filtered = re.sub(r'\s+', ' ', filtered)
        filtered = filtered.strip()
    else:
        filtered = filtered.replace(' ', '')

    return filtered

In [39]:
def compute_symbol_frequencies(text: str) -> Counter:

    return Counter(text)

def compute_bigram_frequencies(text: str, step: int = 1) -> Counter: #step=1 перекривні, step=2 неперекривні

    bigrams = []
    for i in range(0, len(text) - 1, step):
        bg = text[i:i+2]
        if len(bg) == 2:
            bigrams.append(bg)
    return Counter(bigrams)

In [40]:
def compute_entropy_symbol(freq_counter: Counter) -> float:

    total_count = sum(freq_counter.values())
    if total_count == 0:
        return 0.0
    entropy = 0.0
    for _, count in freq_counter.items():
        p = count / total_count
        entropy -= p * math.log2(p)
    return entropy

def compute_entropy_bigram(freq_bigram: Counter) -> float:

    total_count = sum(freq_bigram.values())
    if total_count == 0:
        return 0.0
    entropy = 0.0
    for _, count in freq_bigram.items():
        p = count / total_count
        entropy -= p * math.log2(p)
    return entropy


In [41]:
def compute_redundancy(entropy_symbol: float, alphabet_size: int) -> float:

    if alphabet_size <= 1:
        return 0.0
    max_entropy = math.log2(alphabet_size)
    R = 1 - (entropy_symbol / max_entropy)
    return R

In [42]:
def save_bigram_frequencies_to_csv(freq_bigram: Counter, filename: str):

    total_count = sum(freq_bigram.values())
    sorted_items = sorted(freq_bigram.items(), key=lambda x: x[1], reverse=True)

    with open(filename, 'w', encoding='utf-8') as f:
        f.write("Біграма, Кількість, Частота\n")
        for bg, cnt in sorted_items:
            p = cnt / total_count

            bg_escaped = bg.replace(' ', '[space]')
            f.write(f"{bg_escaped},{cnt},{p:.8f}\n")

def save_symbol_frequencies_to_csv(freq_counter: Counter, filename: str):

    total_count = sum(freq_counter.values())
    sorted_items = sorted(freq_counter.items(), key=lambda x: x[1], reverse=True)

    with open(filename, 'w', encoding='utf-8') as f:
        f.write("Символ, Кількість, Частота\n")
        for sym, cnt in sorted_items:
            p = cnt / total_count
            sym_escaped = '[space]' if sym == ' ' else sym
            f.write(f"{sym_escaped},{cnt},{p:.8f}\n")

In [43]:
def labr():

    filename = "rus_text.txt"

    with open(filename, 'r', encoding='utf-8') as f:
        raw_text = f.read()


    # 1) ТЕКСТ З ПРОБІЛАМИ

    text_ws = filter_text(raw_text, keep_space=True)
    freq_letters_ws = compute_symbol_frequencies(text_ws)

    H1_ws = compute_entropy_symbol(freq_letters_ws)
    m_ws = len(freq_letters_ws) if len(freq_letters_ws) > 0 else 1

    # біграми (перекривні) 
    freq_bigrams_ws_ov = compute_bigram_frequencies(text_ws, step=1)
 
    H2_ws_ov = compute_entropy_bigram(freq_bigrams_ws_ov)
    H2_ws_ov_symbol = H2_ws_ov / 2

    R_bigrams_ws_ov = compute_redundancy(H2_ws_ov_symbol, m_ws)
    save_bigram_frequencies_to_csv(freq_bigrams_ws_ov, "bigrams_with_spaces_overlapping.csv")

    # біграми (неперекривні) 
    freq_bigrams_ws_nonov = compute_bigram_frequencies(text_ws, step=2)

    H2_ws_nonov = compute_entropy_bigram(freq_bigrams_ws_nonov)
    H2_ws_nonov_symbol = H2_ws_nonov / 2

    R_bigrams_ws_nonov = compute_redundancy(H2_ws_nonov_symbol, m_ws)
    save_bigram_frequencies_to_csv(freq_bigrams_ws_nonov, "bigrams_with_spaces_nonoverlapping.csv")

    # частоти символів (з пробілами)
    save_symbol_frequencies_to_csv(freq_letters_ws, "frequencies_with_spaces.csv")


    R1_ws = compute_redundancy(H1_ws, m_ws)


    # 2) ТЕКСТ БЕЗ ПРОБІЛІВ

    text_ns = filter_text(raw_text, keep_space=False)
    freq_letters_ns = compute_symbol_frequencies(text_ns)
    
    H1_ns = compute_entropy_symbol(freq_letters_ns)
    m_ns = len(freq_letters_ns) if len(freq_letters_ns) > 0 else 1

    # біграми (перекривні)
    freq_bigrams_ns_ov = compute_bigram_frequencies(text_ns, step=1)

    H2_ns_ov = compute_entropy_bigram(freq_bigrams_ns_ov)
    H2_ns_ov_symbol = H2_ns_ov / 2

    R_bigrams_ns_ov = compute_redundancy(H2_ns_ov_symbol, m_ns)
    save_bigram_frequencies_to_csv(freq_bigrams_ns_ov, "bigrams_no_spaces_overlapping.csv")

    # біграми (неперекривні)
    freq_bigrams_ns_nonov = compute_bigram_frequencies(text_ns, step=2)

    H2_ns_nonov = compute_entropy_bigram(freq_bigrams_ns_nonov)
    H2_ns_nonov_symbol = H2_ns_nonov / 2
    
    R_bigrams_ns_nonov = compute_redundancy(H2_ns_nonov_symbol, m_ns)
    save_bigram_frequencies_to_csv(freq_bigrams_ns_nonov, "bigrams_no_spaces_nonoverlapping.csv")

    # частоти символів (без пробілів)
    save_symbol_frequencies_to_csv(freq_letters_ns, "frequencies_no_spaces.csv")

    R1_ns = compute_redundancy(H1_ns, m_ns)


    # 3) res.txt

    with open("res.txt", 'w', encoding='utf-8') as out:
        out.write("=== ТЕКСТ З ПРОБІЛАМИ ===\n")
        out.write(f"Довжина тексту з пробілами: {len(text_ws)}\n")
        out.write(f"Кількість унікальних символів: {m_ws}\n")
        out.write(f"H1   = {H1_ws:.6f}\n")
        out.write(f"R    = {R1_ws:.6f}\n\n")

        out.write("--- Біграми (перекривні) ---\n")
        out.write(f"H2 (перекривні)   = {H2_ws_ov_symbol:.6f}\n")
        out.write(f"R  = {R_bigrams_ws_ov:.6f}\n\n")

        out.write("--- Біграми (неперекривні) ---\n")
        out.write(f"H2 (неперекривні)  = {H2_ws_nonov_symbol:.6f}\n")
        out.write(f"R   = {R_bigrams_ws_nonov:.6f}\n\n")

        out.write("============================================================\n")
        out.write("=== ТЕКСТ БЕЗ ПРОБІЛІВ ===\n")
        out.write(f"Довжина тексту без пробілів: {len(text_ns)}\n")
        out.write(f"Кількість унікальних символів: {m_ns}\n")
        out.write(f"H1    = {H1_ns:.6f}\n")
        out.write(f"R    = {R1_ns:.6f}\n\n")

        out.write("--- Біграми (перекривні) ---\n")
        out.write(f"H2 (перекривні)    = {H2_ns_ov_symbol:.6f}\n")
        out.write(f"R  = {R_bigrams_ns_ov:.6f}\n\n")

        out.write("--- Біграми (неперекривні) ---\n")
        out.write(f"H2 (неперекривні)  = {H2_ns_nonov_symbol:.6f}\n")
        out.write(f"R  = {R_bigrams_ns_nonov:.6f}\n\n")

labr()